# Quarta lezione

In [1]:
# This cell loads some html style files (it may be either run/ignored/deleted)
from IPython.core.display import HTML
with open( '../lezioni/style/custom.css', 'r' ) as f: html_style = f.read()
HTML( html_style )

Leggiamo la stessa tabella delle lezioni scorse. 

Specifichiamo al momento della lettura che la colonna `date` contiene dei valori temporali.

Leggo solo la colonna `soil` che contiene la temperatura media giornaliera (misurata nel terreno a 10cm di profondità).

Attenzione che mancano alcuni valori. Per i valori mancanti nella tablella è stato inserito uno spazio. 

In [2]:
import pandas as pd
df = pd.read_csv('../dati/Cork_Airport.csv', 
                 skiprows=24,
                 usecols=['date', 'maxtp', 'mintp', 'soil'],
                 parse_dates=['date'], # Questa colonna sono date
                 na_values = ' ',      # Lo spazio indica valori mancanti
                )
df.head(2)

,date,maxtp,mintp,soil
0,1962-01-01,2.8,-2.5,NaN
1,1962-01-02,2.8,-3.7,NaN


<hr>
__Digressione__ 

La colonna (senza nome) alla sinistra della tabella è un indice numerico progressivo che `Pandas` aggiunge per comodità.

Nel notro caso potrebbe essere naturale usare le date come indice.

In [3]:
df = df.set_index('date')
df.tail(3)

,maxtp,mintp,soil
date,,,
2019-01-29,4.0,-2.0,3.975
2019-01-30,3.4,-2.1,2.600
2019-01-31,6.4,1.3,3.300


Riportiamo il dataframe della condizione originaria.

In [4]:
df = df.reset_index()
df.tail(3)

,date,maxtp,mintp,soil
20847,2019-01-29,4.0,-2.0,3.975
20848,2019-01-30,3.4,-2.1,2.600
20849,2019-01-31,6.4,1.3,3.300


__Fine digressione__

<hr>

Per ognuno dei 12 mesi dell'anno voglio calcolare la temperatura media (media fatta sui giorni del mese e tutti gli anni della tabella). Quindi creare una tabella con 12 righe (una per mese) e colonne per la media.

Per prima cosa aggiungiamo allla tabella una colonna `month` (non è necessario ma è più semplice).

In [5]:
df['month'] = df['date'].map(lambda x: x.month)
df.tail(2)

,date,maxtp,mintp,soil,month
20848,2019-01-30,3.4,-2.1,2.6,1
20849,2019-01-31,6.4,1.3,3.3,1


Conosciamo la seguente procedura per ottenere la media dei valori nel mese di gennaio.

In [6]:
mask = df['month'] == 1
gennaio = df[mask]
gennaio['soil'].mean()

5.1453027730616867

Abbiamo le seguenti 3 possibilità (in ordine di ragionevolezza):
    
*   Ripetere manualmente 12 volte la procedura vista per gennaio. 

*   Scrivere un programmino che automatizza la procedura.

*   Utilizzare il metodo `groupby()` 

Il methodo `groupby()` produce un insieme di dataframe. Uno per ogni valore della colonna che riceve come input. L'output a un formato proprio  `DataFrameGroupBy object`. Non è "printabile" (troppo complesso).

In [7]:
gby = df.groupby('month')
gby

Il metodo `mean()` dell'oggetto `DataFrameGroupBy` si calcola la media dei valori delle colonne dei dataframe "contenuti" nell'ogetto.

In [8]:
aggregate = gby.mean()
aggregate = aggregate[ ['soil'] ] # tengo solo colonne interessanti
aggregate.columns = ['avg_tp']    # cambio nome alle colonne
aggregate

,avg_tp
month,
1,5.145303
2,5.253761
3,6.685498
4,9.115089
5,12.348474
6,15.465327
7,16.937229
8,16.265521
9,14.027778


Il metodo `std()` funziona in modo simile a `mean()`, ma calcola la deviazione standard.

In [9]:
std =  gby.std()
std =  std[ ['soil'] ] # tengo solo colonne interessanti
std.columns = ['std']  # cambio nome alle colonne
std

,std
month,
1,2.251108
2,2.068849
3,1.888505
4,1.933698
5,2.129595
6,2.160759
7,1.937927
8,1.691441
9,1.720731


Ora uso la funzione di Pandas `concat()` per concatenare i due dataframe in uno unico.

In [10]:
aggregate_std = pd.concat( [aggregate,std], axis=1 )
aggregate_std

,avg_tp,std
month,,
1,5.145303,2.251108
2,5.253761,2.068849
3,6.685498,1.888505
4,9.115089,1.933698
5,12.348474,2.129595
6,15.465327,2.160759
7,16.937229,1.937927
8,16.265521,1.691441
9,14.027778,1.720731
